In [1]:
using Gurobi
using QPSReader
using QuadraticModels
using NLPModels
using SolverTools
using SolverBenchmark
using DataFrames

In [2]:
show_statuses()

STATUSES:
  :acceptable     => solved to within acceptable tolerances
  :exception      => unhandled exception
  :first_order    => first-order stationary
  :infeasible     => problem may be infeasible
  :max_eval       => maximum number of function evaluations
  :max_iter       => maximum iteration
  :max_time       => maximum elapsed time
  :neg_pred       => negative predicted reduction
  :not_desc       => not a descent direction
  :small_residual => small residual
  :small_step     => step too small
  :stalled        => stalled
  :unbounded      => objective function may be unbounded from below
  :unknown        => unknown
  :user           => user-requested stop


In [3]:
STATUSES

UndefVarError: UndefVarError: STATUSES not defined

In [121]:
##################################################################
####################### INFO ATTRIBUTES #########################
# https://www.gurobi.com/documentation/9.0/refman/attributes.html 

function createQuadraticModel(qpdata)
    # probleme du point initial
    #x0 = init_x0(qpdata.lvar, qpdata.uvar)
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
                        Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
                        lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
                        c0=qpdata.c0)

end

    

function OptimizeGurobi(QM)
    SM = SlackModel(QM)
    env = Gurobi.Env()
    
    # -1=automatic, 0=primal simplex, 1=dual simplex, 2=barrier, 
    # 3=concurrent, 4=deterministic concurrent, 5=deterministic concurrent simplex.
    setparam!(env, "Method", 2)
    
    # set presolve to 0
    setparam!(env, "Presolve", 0)
    
    model = gurobi_model(env; f = grad(SM, zeros(length(SM.meta.x0))),
                        Aeq = jac(SM, SM.meta.x0), beq = SM.meta.lcon, 
                        lb = SM.meta.lvar, ub = SM.meta.uvar)
     # run optimization
    optimize(model)
    
    # y with dual: b'*y   s.t. A'*y <= c and y >= 0
    y = zeros(size(jac(SM, SM.meta.x0))[1])
    for i=1:size(jac(SM, SM.meta.x0))[1]
        y[i] = Gurobi.get_dblattrelement(model, "Pi", i)
    end

    
    # outputs
    optim_info = get_optiminfo(model)
    if optim_info.status == :optimal
        status = :acceptable
    elseif optim_info.status == :iteration_limit
        status = :max_iter
    elseif optim_info.status == :unbounded 
        status = :unbounded 
    else
        status = :unknown
    end
    stats = GenericExecutionStats(status, SM, solution = get_solution(model),
                                  objective = get_objval(model), iter = optim_info.iter_count,
                                  primal_feas = Gurobi.get_dblattr(model, "ConstrResidual"), # max unscaled primal constraint error
                                  dual_feas = Gurobi.get_dblattr(model, "DualResidual"), # max unscaled dual constraint error
                                  multipliers = y,
                                  elapsed_time = optim_info.runtime)

    return stats
end

function OptimizeGurobi(qpdata::QPSData)
    return OptimizeGurobi(createQuadraticModel(qpdata))
end



OptimizeGurobi (generic function with 2 methods)

In [123]:
path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib"
pb2 = string(path_pb, "\\AFIRO.SIF")
qpdata2 = readqps(pb2);

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [124]:
stats2 = OptimizeGurobi(qpdata2)
println(stats2)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 27 rows, 51 columns and 102 nonzeros
Model fingerprint: 0x84b9128a
Coefficient statistics:
  Matrix range     [1e-01, 2e+00]
  Objective range  [3e-01, 1e+01]
  Bounds range     [8e+01, 5e+02]
  RHS range        [4e+01, 4e+01]
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 6.300e+01
 Factor NZ  : 2.180e+02
 Factor Ops : 2.146e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.31019749e+02  0.00000000e+00  1.57e+02 6.00e-01  1.24e+02     0s
   1   4.58476998e+01 -1.66400779e+02  3.56e+01 9.40e-01  3.13e+01     0s
   2  -9.55565279e+01 -9.76580948e+01  8.19e-03 2.91e-01  1.01e+01     0s
   3  -2.33325835e+02 -4.37125850e+02  3.67e-03 5.45e-02  6.48e+00     0s
   4  -4.58278561e+02 -4.72645999e+02  4.28e-07 2.92e-03  4.5

# Test sur tous les pb

In [24]:
i = 0
for file_name in readdir(path_pb)
    if file_name[end-3:end] == ".SIF" && !(file_name in ["80BAU3B.SIF" ; "BORE3D.SIF"; 
                                                        "CAPRI.SIF"; "CZPROB.SIF"; 
                                                        "ETAMACRO.SIF"; "FINNIS.SIF";
                                                        "FORPLAN.SIF"; "GREENBEA.SIF";
                                                        "GREENBEB.SIF"; "MAROS.SIF";
                                                        "NESM.SIF"; "PEROLD.SIF";
                                                         "PILOT-JA.SIF"; "PILOT-WE.SIF";
                                                         "PILOT.SIF"; "PILOT4.SIF";
                                                         "PILOT87.SIF"; "PILOTNOV.SIF";
                                                          "RECIPELP.SIF"; "SHELL.SIF";
                                                         "SIERRA.SIF"; "STAIR.SIF";
                                                         "STANDATA.SIF"; "STANDGUB.SIF";
                                                        "STANDMPS.SIF"; "TUFF.SIF";
                                                        "VTP-BASE.SIF"]) # problems with fixed variables
        
        i += 1
        println(file_name)
        pb_i = string(path_pb, "\\", file_name)
        if file_name in ["BLEND.SIF"; "DFL001.SIF"; "FORPLAN.SIF"; "GFRD-PNC.SIF"; "SIERRA.SIF"]
            qpdata_i = readqps(pb_i, mpsformat=:fixed) 
        else
            qpdata_i = readqps(pb_i) 
        end
        SM_i = SlackModel(createQuadraticModel(qpdata_i))
        if SM_i.meta.ifix != []
            println("problem fixed variables ", SM_i.meta.ifix, "   ", file_name)
        end
        #model_i = OptimizeGurobi(qpdata_i);

    end
    if i == 1000
        break
        break
    end
end

25FV47.SIF


┌ Info: Using '25FV47' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'R0000' as objective (l. 21)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '.00001' as RHS (l. 6793)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


ADLITTLE.SIF
AFIRO.SIF
AGG.SIF
AGG2.SIF


┌ Info: Using 'ADLITTLE' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using '.Z....' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'ZZZZ0001' as RHS (l. 333)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AGG' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 509)
└ @ QPSReader C:\Users\Geoffroy Leconte\.jul

AGG3.SIF
BANDM.SIF
BEACONFD.SIF
BLEND.SIF
BNL1.SIF


...

┌ Info: Using 'RHS' as RHS (l. 94460)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'TRUSS' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJ' as objective (l. 1022)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350


WOOD1P.SIF


┌ Info: Using 'RHS' as RHS (l. 23745)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'WOOD1P' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751


WOODW.SIF


┌ Info: Using 'COST' as objective (l. 262)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS' as RHS (l. 35551)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'WOODW' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'TRCOST' as objective (l. 1112)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS' as RHS (l. 21608)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [105]:
for file_name in readdir(path_pb)
    println("problem ", file_name)
    
end

problem 25FV47.SIF
problem 80BAU3B.SIF
problem ADLITTLE.SIF
problem AFIRO.SIF
problem AGG.SIF
problem AGG2.SIF
problem AGG3.SIF
problem BANDM.SIF
problem BEACONFD.SIF
problem BLEND.SIF
problem BNL1.SIF
problem BNL2.SIF
problem BOEING1.SIF
problem BOEING2.SIF
problem BORE3D.SIF
problem BRANDY.SIF
problem CAPRI.SIF
problem CRE-A.SIF
problem CRE-B.SIF
problem CRE-C.SIF
problem CRE-D.SIF
problem CYCLE.SIF
problem CZPROB.SIF
problem D2Q06C.SIF
problem D6CUBE.SIF
problem DEGEN2.SIF
problem DEGEN3.SIF
problem DFL001.SIF
problem E226.SIF
problem ETAMACRO.SIF
problem FFFFF800.SIF
problem FINNIS.SIF
problem FIT1D.SIF
problem FIT1P.SIF
problem FIT2D.SIF
problem FIT2P.SIF
problem FORPLAN.SIF
problem GANGES.SIF
problem GFRD-PNC.SIF
problem GREENBEA.SIF
problem GREENBEB.SIF
problem GROW15.SIF
problem GROW22.SIF
problem GROW7.SIF
problem ISRAEL.SIF
problem KB2.SIF
problem KEN-07.SIF
problem KEN-11.SIF
problem KEN-13.SIF
problem KEN-18.SIF
problem LOTFI.SIF
problem MAROS-R7.SIF
problem MAROS.SIF
probl

In [16]:
T = typeof(SM1.meta)
for (name, typ) in zip(fieldnames(T), T.types)
    println("type of the fieldname $name is $typ")
end

type of the fieldname nvar is Int64
type of the fieldname x0 is Array{T,1} where T
type of the fieldname lvar is Array{T,1} where T
type of the fieldname uvar is Array{T,1} where T
type of the fieldname ifix is Array{Int64,1}
type of the fieldname ilow is Array{Int64,1}
type of the fieldname iupp is Array{Int64,1}
type of the fieldname irng is Array{Int64,1}
type of the fieldname ifree is Array{Int64,1}
type of the fieldname iinf is Array{Int64,1}
type of the fieldname nbv is Int64
type of the fieldname niv is Int64
type of the fieldname nlvb is Int64
type of the fieldname nlvo is Int64
type of the fieldname nlvc is Int64
type of the fieldname nlvbi is Int64
type of the fieldname nlvci is Int64
type of the fieldname nlvoi is Int64
type of the fieldname nwv is Int64
type of the fieldname ncon is Int64
type of the fieldname y0 is Array{T,1} where T
type of the fieldname lcon is Array{T,1} where T
type of the fieldname ucon is Array{T,1} where T
type of the fieldname jfix is Array{Int64,1